# Extracting HuBERT Hidden Representations

In [1]:
# Packages
from transformers import HubertForCTC, Wav2Vec2Processor
import numpy as np
import glob
import os
from collections import defaultdict 

# Custom Helper Libraries
from helper_scripts.TenseLax import TenseLax
from helper_scripts.AudioProcessing import AudioProcessing
from helper_scripts.Constants import *
from helper_scripts.Pathing import Pathing

/Users/kyleng/B_Organized/A_School/Ling_487/clean_code/Probe-HuBERT/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/kyleng/B_Organized/A_School/Ling_487/clean_code/Probe-HuBERT/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at facebook/hubert-large-ls960-ft were not used when initializing HubertForCTC: ['hubert.encoder.pos_conv_embed.conv.weight_g', 'hubert.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertForCTC from the checkpoint of a model trained on another task or with another archit

# Roadmap
```
let n = number of samples of audio files
let s = number of segments/phonemes to be segmented
let k = number of encoders in LLM
let l[i] = number of speech vectors per input sequence
```

1) Select n=200 samples of audio files from a specific subset of dialects from `TIMIT`
2) Create the output `hidden_states`
    - s entries
    - each entry is a size k=25 array 
    - each array index is a numpy array representing `l[i]` speech vectors--where each speech vector is of size 1024
3) Load in each of the n samples using `librosa`
4) 

# 1) Import Audio Files
- Extract 200 path's to audio samples to save computation

In [2]:
# Path is TIMIT/<TEST or TRAIN>/<DIALECT>/<SPEAKER ID>/<SEGMENT ID>.wav
DATASET_PATH = "../Timit-Database/TIMIT/"
ALL_WAVS_PATH = os.path.join(DATASET_PATH, "*", "*", "*", "*.wav")

speech_paths = glob.glob(ALL_WAVS_PATH)
print(f"Importing {len(speech_paths)} speech samples")

speech_paths = AudioProcessing.select_samples(
    speech_paths,
    num_samples=Constants.EXPERIMENTATION.NUM_SPEECH_SAMPLES
)

print(f"Succesfully randomly sampled {len(speech_paths)} speech samples")

Importing 4969 speech samples
Succesfully randomly sampled 200 speech samples


# 2) Create the Data Structure for Saving the Hidden States

```
For Each Phoneme
    For Each Encoder 
        For Each Sequence of 1024 Vectors
            Append Hidden State
```

Hidden States maps <`phoneme string`, a list of`Hidden State Representation`>

In [3]:
all_hidden_states = defaultdict(list)

# 2) Calculate Boundaries for each Audio File

In [4]:
for path in speech_paths:
    # Step 1) Generate the hidden states and boundaries
    embedded_audio, num_speech_frames, sequence_length = AudioProcessing.process_audio(
        wav_path=path,
        embedding_model=Constants.EXPERIMENTATION.EMBEDDING_MODEL,
        inference_model=Constants.EXPERIMENTATION.INFERENCE_MODEL,
        sampling_rate=16000
    )

    scaled_segmentation = AudioProcessing.get_sequence_boundary(
        TIMIT_wav_path=path,
        num_speech_frames=num_speech_frames,
        num_speech_vec=sequence_length
    )

    # Step 2) Select boundaries for matching phonemes
    filtered_segmentation = AudioProcessing.filter_segmentation(
        combined_df=scaled_segmentation,
        desired_phonemes=TenseLax.getSet()
    )

    # Step 3) Place Hidden State into output matrix
    for row in filtered_segmentation.itertuples():
        _, seq_start_vec_idx, seq_end_vec_idx, phoneme = row
        
        # Step 3a) Get the Hidden States per encoder for the entire speech segment
        utterance_hidden_states = AudioProcessing.get_hidden_states(
            input_embedding=embedded_audio,
            inference_model=Constants.EXPERIMENTATION.INFERENCE_MODEL,
            start_idx=seq_start_vec_idx,
            end_idx=seq_end_vec_idx 
        )

        # Step 3b) Append hidden States to the existing hidden states for this row
        all_hidden_states[phoneme].append(
            utterance_hidden_states
        )
    

# 3) Save Phoneme Hidden States

In [8]:
for phoneme, hidden_state in all_hidden_states.items():
    combined_per_segment = np.concatenate(hidden_state, axis=1)
    print(f"{phoneme}: {combined_per_segment.shape}")
    Pathing.save_file_np(
        save_dir=Constants.PATHING.hidden_state_save_path,
        save_file_name=f"HS_{phoneme}_{combined_per_segment.shape[1]}.npy",
        to_save=combined_per_segment
    )

ao: (25, 857, 1024)
ey: (25, 635, 1024)
iy: (25, 1453, 1024)
ae: (25, 1483, 1024)
ow: (25, 555, 1024)
eh: (25, 790, 1024)
ih: (25, 877, 1024)
uh: (25, 62, 1024)
uw: (25, 188, 1024)
